In [ ]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
from renderer import SubGridSpecRenderer, GridSpecRenderer, AxesRenderer, \
PlotRenderer, render
import os
from benchart import BenchArt, Run
from metadata import RunMetadata
from loader import Loader

def discard_large_read(all_data):
    large_read = all_data['metadata']['benchmark'].get('large_read', None)
    if large_read is None or large_read == '' or large_read == 'none':
        return False
    return True

def data_def(all_data):
    return all_data['data']['pgbench']['progress']

def metadata_def(all_data):
    return all_data['metadata']

loader = Loader('run_data2')
loader.discard(discard_large_read)
runs = loader.run(data_def, metadata_def)

benchart = BenchArt(runs)

relabels = {
    'machine_instance_hostinfo_Hostname': 'host',
    'benchmark_config_scale': 'scale',
    'machine_instance_hostinfo_KernelRelease': 'kernel',
    'machine_disk_block_device_settings_nr_hw_queues': '#hwq',
    'machine_disk_block_device_settings_nr_requests': 'nr_requests',
    'machine_disk_block_device_settings_queue_depth': 'queue_depth',
    'machine_disk_block_device_settings_scheduler': 'scheduler',
    'postgres.gucs.set_gucs.huge_pages': 'huge_pages',
}

figure = plt.figure(figsize=(15,40), constrained_layout=True)


disk_identifier = ['machine_disk_limits_burst_iops',
                   'machine_disk_limits_iops', 'machine_disk_limits_size',
                   'machine_disk_limits_burst_bw_mbps',
                   'machine_disk_limits_bw_mbps', 'machine_disk_size_gb', ]

benchart.part(SubGridSpecRenderer(figure, relabels), 'machine_instance_hostinfo_Hostname',
              'machine_instance_type', *disk_identifier)


benchart.part(SubGridSpecRenderer(figure, relabels), 'benchmark_config_scale')
benchart.part(SubGridSpecRenderer(figure, relabels), 'benchmark_large_read')

benchart.ignore('machine_instance_hostinfo_KernelRelease',
                'machine_disk_block_device_settings_nr_hw_queues',
                'machine_disk_block_device_settings_nr_requests',
                'machine_disk_block_device_settings_queue_depth',
                'machine_disk_block_device_settings_scheduler',
                'postgres.gucs.set_gucs.huge_pages')

gs = render(benchart, figure, relabels)
